# Fine tune language models

In [1]:
from argparse import Namespace
import copy
import pandas as pd
from gensim.models import Word2Vec
from gensim import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#nlp = spacy.load("en_core_web_lg")

## Initialize a pre-trained language model

In [3]:
# model types
# w2v: word2vec
# ft: fasttext
model_type = "w2v"   
model_path = "/Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model"

if model_type.lower() in ["w2v", "word2vec"]:
    # Word2Vec
    embedding_model_orig = Word2Vec.load(model_path)
elif model_type.lower() in ["ft", "fasttext"]:
    # FastText
    embedding_model_orig = FastText.load(model_path)
print(embedding_model_orig)

2019-11-20 09:40:47,233 : INFO : loading Word2Vec object from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model
2019-11-20 09:40:48,248 : INFO : loading wv recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.* with mmap=None
2019-11-20 09:40:48,249 : INFO : loading vectors from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.vectors.npy with mmap=None
2019-11-20 09:40:48,544 : INFO : setting ignored attribute vectors_norm to None
2019-11-20 09:40:48,545 : INFO : loading vocabulary recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/m

Word2Vec(vocab=434049, size=300, alpha=0.03)


In [4]:
min_alpha_yet_reached = embedding_model_orig.min_alpha_yet_reached
min_alpha_yet_reached

0.0007000009816451012

## Read DB

In [5]:
db_sentence_orig = pd.read_pickle("./db_trove_v002.pkl")

# Quality bands 3, 4

In [6]:
db_sentence = db_sentence_orig[(db_sentence_orig['quality_band'] == 3) | 
                               (db_sentence_orig['quality_band'] == 4)]
db_sentence['quality_band'].value_counts()

3    3600
4    1495
Name: quality_band, dtype: int64

## preprocess DB

In [7]:
"""
def cleanup(myrow, colname="corrected"):
    # remove all # and @§
    
    corpus = [re.sub(r'#', '', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [re.sub(r'@', '', element, flags=re.IGNORECASE) for element in corpus]
    
    # --- remove 2 or more .
    corpus = [re.sub('[.]{2,}', '.', element) for element in corpus]
    # --- add a space before and after a list of punctuations
    corpus = [re.sub(r"([.,!?:;\"\'])", r" \1 ", element) for element in corpus]
    # --- remove everything except:
    #corpus = [re.sub(r"([^a-zA-Z\-.:;,!?\d+]+)", r" ", element) for element in corpus]
    corpus = [re.sub(r"([^a-zA-Z\d+]+)", r" ", element) for element in corpus]
    # --- replace numbers with <NUM>
    corpus = [re.sub(r'\b\d+\b', '<NUM>', element) for element in corpus]
    corpus = [re.sub('--', '', element) for element in corpus]
    # --- normalize white spaces
    corpus = [re.sub('\s+', ' ', element) for element in corpus]
    
    # remove multiple spaces
    corpus = [re.sub(r'\s+', ' ', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [element.strip() for element in corpus]
    #corpus = [element.lower() for element in corpus]
    return corpus
"""

'\ndef cleanup(myrow, colname="corrected"):\n    # remove all # and @§\n    \n    corpus = [re.sub(r\'#\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    corpus = [re.sub(r\'@\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    \n    # --- remove 2 or more .\n    corpus = [re.sub(\'[.]{2,}\', \'.\', element) for element in corpus]\n    # --- add a space before and after a list of punctuations\n    corpus = [re.sub(r"([.,!?:;"\'])", r" \x01 ", element) for element in corpus]\n    # --- remove everything except:\n    #corpus = [re.sub(r"([^a-zA-Z\\-.:;,!?\\d+]+)", r" ", element) for element in corpus]\n    corpus = [re.sub(r"([^a-zA-Z\\d+]+)", r" ", element) for element in corpus]\n    # --- replace numbers with <NUM>\n    corpus = [re.sub(r\'\x08\\d+\x08\', \'<NUM>\', element) for element in corpus]\n    corpus = [re.sub(\'--\', \'\', element) for element in corpus]\n    # --- normalize white spaces\n    corpus = [re.sub(\'\\s+\', \' \', element) for elem

In [8]:
def cleanup(myrow, col_name):
    all_clean_rows = []
    for sent in myrow[col_name]:
        one_clean_row = []
        for token in sent:
            one_clean_row.append(token.lower())
        all_clean_rows.append(one_clean_row)
    return all_clean_rows

In [9]:
db_sentence["ocr_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["ocr_sentencizer"], axis=1)
db_sentence["corrected_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["corrected_sentencizer"], axis=1)

/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
print(db_sentence.shape)
db_sentence.head()

(5095, 25)


,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,...,ocr_dict_lookup,ocr_dict_lookup_list,ocr_dict_perc,corrected_dict_lookup_list,corr_dict_perc,corrected_sentencizer_list,ocr_sentencizer_list,jaccard_similarity,ocr_sentencizer_cleaned,corrected_sentencizer_cleaned
3,./trove_overproof/datasets/dataset1/rawTextAnd...,18366055,Article,1953,FIRST CHURCH I SERVICE 1 Presbyterian I ' Anni...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,0.738901,946,832,...,"[[5, 6, 1, 7, 1, 12, 1, 1, 11, 1, 3, 5, -12, 3...","[5, 6, 1, 7, 1, 12, 1, 1, 11, 1, 3, 5, -12, 3,...",82.000000,"[5, 6, 7, 12, 11, 3, 5, 11, 2, 3, 5, 12, 6, 7,...",99.206349,"[FIRST, CHURCH, SERVICE, Presbyterian, Anniver...","[FIRST, CHURCH, I, SERVICE, 1, Presbyterian, I...",0.191919,"[[first, church, i, service, 1, presbyterian, ...","[[first, church, service, presbyterian, annive..."
4,./trove_overproof/datasets/dataset1/rawTextAnd...,18386137,Article,1953,"""Bob"" Lulham's Fight Against Thallium District...","""Bob"" Lulham's Fight Against Thallium Arthur ...","""Bob"" Lulham's Fight Against Thallium Arthur ...",0.493898,2950,2740,...,"[[1, 3, 1, -6, 2, 5, 7, 8, 8, 8, 1], [4, 5, 6,...","[1, 3, 1, -6, 2, 5, 7, 8, 8, 8, 1, 4, 5, 6, 6,...",95.580110,"[1, 3, 1, -6, 2, 5, 7, 8, 6, 6, 1, 1, 3, 1, 1,...",95.652174,"["", Bob, "", Lulham, 's, Fight, Against, Thalli...","["", Bob, "", Lulham, 's, Fight, Against, Thalli...",0.282682,"[["", bob, "", lulham, 's, fight, against, thall...","[["", bob, "", lulham, 's, fight, against, thall..."
10,./trove_overproof/datasets/dataset1/rawTextAnd...,18391223,Article,1953,"rchitect For lympics aid £31,665 - ?? MELBOURN...","Architect For Olympics paid £31,665 MELBOURNE...","Architect For Olympics paid £31,665 MELBOURNE...",0.793510,678,585,...,"[[-8, 3, 7, 3, 1, -6, 1, 2, 9, 1, 6], [1, 1, 5...","[-8, 3, 7, 3, 1, -6, 1, 2, 9, 1, 6, 1, 1, 5, 5...",88.607595,"[9, 3, 8, 4, 1, -6, 9, 1, 6, 2, 5, 5, 1, 3, 9,...",94.680851,"[Architect, For, Olympics, paid, £, 31,665, ME...","[rchitect, For, lympics, aid, £, 31,665, -, ??...",0.302885,"[[rchitect, for, lympics, aid, £, 31,665, -, ?...","[[architect, for, olympics, paid, £, 31,665, m..."
21,./trove_overproof/datasets/dataset1/rawTextAnd...,18392087,Article,1953,"Mint Building i Sur,--Tb* cía .\-fint Build- i...","Mint Building Sir, - The old Mint Build- ing i...","Mint Building Sir, - The old Mint Build- ing i...",0.687574,813,845,...,"[[4, 8, 1, -8, 1, 3, 1, 1, 1, 4, 5, 1, 3, 2, 9...","[4, 8, 1, -8, 1, 3, 1, 1, 1, 4, 5, 1, 3, 2, 9,...",64.347826,"[4, 8, 3, 1, 1, 3, 3, 4, 5, 1, 3, 2, 9, 6, 1, ...",98.333333,"[Mint, Building, Sir, ,, -, The, old, Mint, Bu...","[Mint, Building, i, Sur,--Tb, *, cía, ., \, -,...",0.163399,"[[mint, building, i, sur,--tb, *, cía, ., \, -...","[[mint, building, sir, ,, -, the, old, mint, b..."
25,./trove_overproof/datasets/dataset1/rawTextAnd...,27904257,Article ILLUSTRATED,1947,"PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...","PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...","PEARL TOWN PAINTED BY ELIZABETH DURACK ""The st...",0.744510,2732,2722,...,"[[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4,...","[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4, ...",96.800000,"[5, 4, 7, 2, 9, 6, 1, 3, 5, 2, 3, 5, 4, 8, 4, ...",99.726776,"[PEARL, TOWN, PAINTED, BY, ELIZABETH, DURACK, ...","[PEARL, TOWN, PAINTED, BY, ELIZABETH, DURACK, ...",0.236032,"[[pearl, town, painted, by, elizabeth, durack,...","[[pearl, town, painted, by, elizabeth, durack,..."


## Update a pre-trained LM

In [11]:
# args for Word2Vec
w2v_args = Namespace(
    epochs=5, 
    # only for Word2Vec
    compute_loss=True,                               # If True, computes and stores loss value which can be retrieved using get_latest_training_loss().

#     size=100,                                        # Dimensionality of the word vectors.
#     alpha=0.03,                                      # The initial learning rate.
#     min_alpha=0.0007,                                # Learning rate will linearly drop to min_alpha as training progresses.
#     sg=1,                                            # Training algorithm: skip-gram if sg=1, otherwise CBOW.
#     hs=0,                                            # If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
#     negative=20,                                     # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used. 
#     min_count=5,                                    # The model ignores all words with total frequency lower than this.
#     window=5,                                        # The maximum distance between the current and predicted word within a sentence.
#     sample=1e-3,                                     # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
#     workers=8, 
#     cbow_mean=1,                                     # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
#     null_word=0,                                     # 
#     trim_rule=None,                                  # 
#     sorted_vocab=1,                                  # If 1, sort the vocabulary by descending frequency before assigning word indices.
#     batch_words=10000,                               # Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
    
#     seed=1364,                                       # Seed for the random number generator.
#     # only for FastText (compare to word2vec)
#     #word_ngrams=1,                                   # If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec. 
#     #min_n=2,                                         # Minimum length of char n-grams to be used for training word representations.
#     #max_n=15,                                        # Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
#     #bucket=2000000                                  # Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
)

## Preprocess before creating/updating LM

In [12]:
"""
def preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):
    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]
    return txt
"""

'\ndef preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):\n    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]\n    return txt\n'

In [13]:
"""
db_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)
db_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)
"""

'\ndb_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)\ndb_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)\n'

## OCR

In [14]:
list_sentences = db_sentence["ocr_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 5095


In [15]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [16]:
flattened_list_sentences[0]

['first',
 'church',
 'i',
 'service',
 '1',
 'presbyterian',
 'i',
 "'",
 'anniversary',
 '1',
 'the',
 '150th',
 'anniveisarjj',
 'the',
 'first',
 'presb',
 '>',
 'terian',
 'chad',
 'service',
 'm',
 'australia',
 'will',
 'k',
 'celebrated',
 'to',
 'morrow',
 'special',
 'services',
 'will',
 'be',
 'be',
 'at',
 'the',
 'ebenezer',
 'p«sbjt«j',
 'church',
 ',',
 'six',
 'miles',
 'from',
 'wmdso',
 'the',
 'ebenezer',
 'church',
 'is',
 '»',
 'oldest',
 'church',
 'in',
 'australia',
 '?',
 'which',
 'services',
 'are',
 'still',
 '«',
 'ia',
 'the',
 'sen',
 'ice',
 'v',
 '.',
 '.',
 '11',
 'start',
 'at',
 'lp',
 '£',
 'with',
 'sacred',
 'mus.c',
 'ttajjl',
 '*',
 'followed',
 'at',
 '130',
 'p',
 'm',
 '«*£',
 'dedication',
 'of',
 'commemorate',
 'g',
 'tue',
 'moderator',
 'general',
 'of',
 'j',
 '«',
 'presbyterian',
 'church',
 'of',
 'au',
 '»',
 'the',
 'right',
 'rev',
 '^',
 '¿',
 'bz',
 'will',
 'conduct',
 'a',
 'special',
 'atfhe',
 'pä',
 'ws«.llbe^r',
 'the',
 

In [17]:
embedding_model_ocr = copy.deepcopy(embedding_model_orig)

In [18]:
embedding_model_ocr.workers = 8
embedding_model_ocr.vocabulary.min_count = 5
embedding_model_ocr.alpha = min_alpha_yet_reached*10.

In [19]:
embedding_model_ocr.build_vocab(flattened_list_sentences, update=True)
embedding_model_ocr.train(flattened_list_sentences, 
                          total_examples=embedding_model_ocr.corpus_count,
                          epochs=w2v_args.epochs,  
                          compute_loss=w2v_args.compute_loss)

2019-11-20 09:41:06,575 : INFO : collecting all words and their counts
2019-11-20 09:41:06,576 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 09:41:06,674 : INFO : PROGRESS: at sentence #10000, processed 434822 words, keeping 90410 word types
2019-11-20 09:41:06,791 : INFO : PROGRESS: at sentence #20000, processed 886062 words, keeping 160063 word types
2019-11-20 09:41:06,911 : INFO : PROGRESS: at sentence #30000, processed 1366349 words, keeping 227866 word types
2019-11-20 09:41:07,012 : INFO : PROGRESS: at sentence #40000, processed 1789264 words, keeping 284758 word types
2019-11-20 09:41:07,035 : INFO : collected 298566 word types from a corpus of 1896341 raw words and 42059 sentences
2019-11-20 09:41:07,036 : INFO : Updating model with new vocabulary
2019-11-20 09:41:07,184 : INFO : New added 19776 unique words (6% of original 318342) and increased the count of 19776 pre-existing words (6% of original 318342)
2019-11-20 09:41:07,319 : INFO 

2019-11-20 09:41:49,139 : INFO : EPOCH 4 - PROGRESS: at 90.92% examples, 121600 words/s, in_qsize 14, out_qsize 1
2019-11-20 09:41:49,629 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-11-20 09:41:49,745 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-20 09:41:49,802 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 09:41:49,843 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 09:41:49,871 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 09:41:49,876 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 09:41:49,914 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 09:41:49,920 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 09:41:49,920 : INFO : EPOCH - 4 : training on 1896341 raw words (1177341 effective words) took 9.5s, 124139 effective words/s
2019-11-20 09:41:51,1

(5888314, 9481705)

In [20]:
print("\n\n[INFO] Save the model")
embedding_model_ocr.save("./w2v_005_EM_ocr_qual_3_4.model")

2019-11-20 09:41:59,817 : INFO : saving Word2Vec object under ./w2v_005_EM_ocr_qual_3_4.model, separately None
2019-11-20 09:41:59,817 : INFO : storing np array 'vectors' to ./w2v_005_EM_ocr_qual_3_4.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 09:42:00,553 : INFO : not storing attribute vectors_norm
2019-11-20 09:42:00,554 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_ocr_qual_3_4.model.trainables.syn1neg.npy
2019-11-20 09:42:01,260 : INFO : not storing attribute cum_table
2019-11-20 09:42:02,329 : INFO : saved ./w2v_005_EM_ocr_qual_3_4.model


## Corrected

In [21]:
list_sentences = db_sentence["corrected_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 5095


In [22]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [23]:
flattened_list_sentences[0]

['first',
 'church',
 'service',
 'presbyterian',
 'anniversary',
 'the',
 '150th',
 'anniversary',
 'of',
 'the',
 'first',
 'presbyterian',
 'church',
 'service',
 'in',
 'australia',
 'will',
 'be',
 'celebrated',
 'to',
 'morrow',
 '.']

In [24]:
embedding_model_corrected = copy.deepcopy(embedding_model_orig)

In [25]:
embedding_model_corrected.workers = 8
embedding_model_corrected.vocabulary.min_count = 5
embedding_model_corrected.alpha = min_alpha_yet_reached*10.

In [26]:
embedding_model_corrected.build_vocab(flattened_list_sentences, update=True)
embedding_model_corrected.train(flattened_list_sentences, 
                                total_examples=embedding_model_corrected.corpus_count,
                                epochs=w2v_args.epochs,  
                                compute_loss=w2v_args.compute_loss)

2019-11-20 09:42:10,527 : INFO : collecting all words and their counts
2019-11-20 09:42:10,529 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 09:42:10,577 : INFO : PROGRESS: at sentence #10000, processed 217550 words, keeping 21037 word types
2019-11-20 09:42:10,634 : INFO : PROGRESS: at sentence #20000, processed 455085 words, keeping 31368 word types
2019-11-20 09:42:10,679 : INFO : PROGRESS: at sentence #30000, processed 695671 words, keeping 40549 word types
2019-11-20 09:42:10,721 : INFO : PROGRESS: at sentence #40000, processed 923693 words, keeping 47531 word types
2019-11-20 09:42:10,760 : INFO : PROGRESS: at sentence #50000, processed 1136699 words, keeping 53209 word types
2019-11-20 09:42:10,802 : INFO : PROGRESS: at sentence #60000, processed 1361413 words, keeping 58750 word types
2019-11-20 09:42:10,838 : INFO : PROGRESS: at sentence #70000, processed 1582999 words, keeping 63795 word types
2019-11-20 09:42:10,874 : INFO : collected 

2019-11-20 09:42:50,909 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-11-20 09:42:50,939 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-20 09:42:51,040 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 09:42:51,050 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 09:42:51,114 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-20 09:42:51,188 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-20 09:42:51,194 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-20 09:42:51,206 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 09:42:51,207 : INFO : EPOCH - 4 : training on 1807730 raw words (1215530 effective words) took 8.6s, 141573 effective words/s
2019-11-20 09:42:52,401 : INFO : EPOCH 5 - PROGRESS: at 9.82% examples, 98167 words/s, in_qsize 16, out_qsize 1
2019-11-20 09:42:53,644

(6077696, 9038650)

In [27]:
print("\n\n[INFO] Save the model")
embedding_model_corrected.save("./w2v_005_EM_corr_qual_3_4.model")

2019-11-20 09:43:00,712 : INFO : saving Word2Vec object under ./w2v_005_EM_corr_qual_3_4.model, separately None
2019-11-20 09:43:00,713 : INFO : storing np array 'vectors' to ./w2v_005_EM_corr_qual_3_4.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 09:43:01,413 : INFO : not storing attribute vectors_norm
2019-11-20 09:43:01,413 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_corr_qual_3_4.model.trainables.syn1neg.npy
2019-11-20 09:43:02,225 : INFO : not storing attribute cum_table
2019-11-20 09:43:03,251 : INFO : saved ./w2v_005_EM_corr_qual_3_4.model


# =================================

# Quality bands 1, 2

In [28]:
db_sentence = db_sentence_orig[(db_sentence_orig['quality_band'] == 1) | 
                               (db_sentence_orig['quality_band'] == 2)]
db_sentence['quality_band'].value_counts()

2    13953
1    11461
Name: quality_band, dtype: int64

## preprocess DB

In [29]:
"""
def cleanup(myrow, colname="corrected"):
    # remove all # and @§
    
    corpus = [re.sub(r'#', '', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [re.sub(r'@', '', element, flags=re.IGNORECASE) for element in corpus]
    
    # --- remove 2 or more .
    corpus = [re.sub('[.]{2,}', '.', element) for element in corpus]
    # --- add a space before and after a list of punctuations
    corpus = [re.sub(r"([.,!?:;\"\'])", r" \1 ", element) for element in corpus]
    # --- remove everything except:
    #corpus = [re.sub(r"([^a-zA-Z\-.:;,!?\d+]+)", r" ", element) for element in corpus]
    corpus = [re.sub(r"([^a-zA-Z\d+]+)", r" ", element) for element in corpus]
    # --- replace numbers with <NUM>
    corpus = [re.sub(r'\b\d+\b', '<NUM>', element) for element in corpus]
    corpus = [re.sub('--', '', element) for element in corpus]
    # --- normalize white spaces
    corpus = [re.sub('\s+', ' ', element) for element in corpus]
    
    # remove multiple spaces
    corpus = [re.sub(r'\s+', ' ', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [element.strip() for element in corpus]
    #corpus = [element.lower() for element in corpus]
    return corpus
"""

'\ndef cleanup(myrow, colname="corrected"):\n    # remove all # and @§\n    \n    corpus = [re.sub(r\'#\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    corpus = [re.sub(r\'@\', \'\', element, flags=re.IGNORECASE) for element in corpus]\n    \n    # --- remove 2 or more .\n    corpus = [re.sub(\'[.]{2,}\', \'.\', element) for element in corpus]\n    # --- add a space before and after a list of punctuations\n    corpus = [re.sub(r"([.,!?:;"\'])", r" \x01 ", element) for element in corpus]\n    # --- remove everything except:\n    #corpus = [re.sub(r"([^a-zA-Z\\-.:;,!?\\d+]+)", r" ", element) for element in corpus]\n    corpus = [re.sub(r"([^a-zA-Z\\d+]+)", r" ", element) for element in corpus]\n    # --- replace numbers with <NUM>\n    corpus = [re.sub(r\'\x08\\d+\x08\', \'<NUM>\', element) for element in corpus]\n    corpus = [re.sub(\'--\', \'\', element) for element in corpus]\n    # --- normalize white spaces\n    corpus = [re.sub(\'\\s+\', \' \', element) for elem

In [30]:
def cleanup(myrow, col_name):
    all_clean_rows = []
    for sent in myrow[col_name]:
        one_clean_row = []
        for token in sent:
            one_clean_row.append(token.lower())
        all_clean_rows.append(one_clean_row)
    return all_clean_rows

In [31]:
db_sentence["ocr_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["ocr_sentencizer"], axis=1)
db_sentence["corrected_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["corrected_sentencizer"], axis=1)

/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/khosseini/anaconda3/envs/py37torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
print(db_sentence.shape)
db_sentence.head()

(25414, 25)


,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,...,ocr_dict_lookup,ocr_dict_lookup_list,ocr_dict_perc,corrected_dict_lookup_list,corr_dict_perc,corrected_sentencizer_list,ocr_sentencizer_list,jaccard_similarity,ocr_sentencizer_cleaned,corrected_sentencizer_cleaned
1,./trove_overproof/datasets/dataset1/rawTextAnd...,18378453,Article ILLUSTRATED,1953,"FROM RIVER CROSSING TO END OF TRIÄÜ I ^PI A^H""...",FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,0.847561,746,820,...,"[[4, 5, 8, 2, 3, 2, -5, 1, 1, 2, -3, 1, -5, -6...","[4, 5, 8, 2, 3, 2, -5, 1, 1, 2, -3, 1, -5, -6,...",79.439252,"[4, 5, 8, 2, 3, 2, 5, 6, 1, -5, -6, 8, 4, 4, 5...",92.079208,"[FROM, RIVER, CROSSING, TO, END, OF, TRIAL, SP...","[FROM, RIVER, CROSSING, TO, END, OF, TRIÄÜ, I,...",0.305882,"[[from, river, crossing, to, end, of, triäü, i...","[[from, river, crossing, to, end, of, trial, s..."
2,./trove_overproof/datasets/dataset1/rawTextAnd...,18363627,Article,1953,"Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...",0.964119,641,630,...,"[[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, ...","[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, 2...",96.590909,"[7, 10, -7, 6, 4, 4, 3, 3, 4, 5, 6, 4, 6, 5, 2...",98.876404,"[Natural, Childbirth, Sir,-We, nurses, have, s...","[Natural, Childbirth, Sir,-We, nurses, have, s...",0.523810,"[[natural, childbirth, sir,-we, nurses, have, ...","[[natural, childbirth, sir,-we, nurses, have, ..."
5,./trove_overproof/datasets/dataset1/rawTextAnd...,18368961,Article,1953,"DIVORCE Before The Judge In Divorce, Mr Justic...","DIVORCE Before The Judge In Divorce, Mr. Justi...","DIVORCE Before The Judge In Divorce, Mr. Justi...",0.894176,1219,1121,...,"[[7, 6, 3, 5, 2, 7, 1, 2, 7, -5, 7, 4, 1, 1, -...","[7, 6, 3, 5, 2, 7, 1, 2, 7, -5, 7, 4, 1, 1, -6...",82.882883,"[7, 6, 3, 5, 2, 7, 1, 2, 1, 7, -5, 7, 4, 1, 1,...",86.486486,"[DIVORCE, Before, The, Judge, In, Divorce, ,, ...","[DIVORCE, Before, The, Judge, In, Divorce, ,, ...",0.147513,"[[divorce, before, the, judge, in, divorce, ,,...","[[divorce, before, the, judge, in, divorce, ,,..."
6,./trove_overproof/datasets/dataset1/rawTextAnd...,18355541,Article,1953,"BRITISH AIRLINER I MISSING ¡HALIFAX, Feb. 2 M....","BRITISH AIRLINER MISSING HALIFAX, Feb. 2 (A.A....","BRITISH AIRLINER MISSING HALIFAX, Feb. 2 (A.A....",0.829670,1274,1139,...,"[[7, 8, 1, 7, 1, 7, 1, 3, 1, 1, 4, 1, -3, 7, 3...","[7, 8, 1, 7, 1, 7, 1, 3, 1, 1, 4, 1, -3, 7, 3,...",88.194444,"[7, 8, 7, 7, 1, 3, 1, 1, 1, -6, 1, -3, 7, 3, 1...",95.731707,"[BRITISH, AIRLINER, MISSING, HALIFAX, ,, Feb, ...","[BRITISH, AIRLINER, I, MISSING, ¡, HALIFAX, ,,...",0.240449,"[[british, airliner, i, missing, ¡, halifax, ,...","[[british, airliner, missing, halifax, ,, feb,..."
7,./trove_overproof/datasets/dataset1/rawTextAnd...,18381450,Article,1953,I SCHOOL CHESS * Homebush Increased Ils lead o...,SCHOOL CHESS Homebush increased its lead over...,SCHOOL CHESS Homebush increased its lead over...,0.918264,991,955,...,"[[1, 6, 5, 1, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2,...","[1, 6, 5, 1, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2, ...",92.307692,"[6, 5, 8, 9, 3, 4, 4, 8, 2, 3, 5, 5, 2, 3, 1, ...",96.598639,"[SCHOOL, CHESS, Homebush, increased, its, lead...","[I, SCHOOL, CHESS, *, Homebush, Increased, Ils...",0.277612,"[[i, school, chess, *, homebush, increased, il...","[[school, chess, homebush, increased, its, lea..."


## Update a pre-trained LM

In [33]:
# args for Word2Vec
w2v_args = Namespace(
    epochs=5, 
    # only for Word2Vec
    compute_loss=True,                               # If True, computes and stores loss value which can be retrieved using get_latest_training_loss().

#     size=100,                                        # Dimensionality of the word vectors.
#     alpha=0.03,                                      # The initial learning rate.
#     min_alpha=0.0007,                                # Learning rate will linearly drop to min_alpha as training progresses.
#     sg=1,                                            # Training algorithm: skip-gram if sg=1, otherwise CBOW.
#     hs=0,                                            # If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
#     negative=20,                                     # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used. 
#     min_count=5,                                    # The model ignores all words with total frequency lower than this.
#     window=5,                                        # The maximum distance between the current and predicted word within a sentence.
#     sample=1e-3,                                     # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
#     workers=8, 
#     cbow_mean=1,                                     # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
#     null_word=0,                                     # 
#     trim_rule=None,                                  # 
#     sorted_vocab=1,                                  # If 1, sort the vocabulary by descending frequency before assigning word indices.
#     batch_words=10000,                               # Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
    
#     seed=1364,                                       # Seed for the random number generator.
#     # only for FastText (compare to word2vec)
#     #word_ngrams=1,                                   # If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec. 
#     #min_n=2,                                         # Minimum length of char n-grams to be used for training word representations.
#     #max_n=15,                                        # Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
#     #bucket=2000000                                  # Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
)

## Preprocess before creating/updating LM

In [34]:
"""
def preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):
    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]
    return txt
"""

'\ndef preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):\n    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]\n    return txt\n'

In [35]:
"""
db_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)
db_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)
"""

'\ndb_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)\ndb_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)\n'

## OCR

In [36]:
list_sentences = db_sentence["ocr_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 25414


In [37]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [38]:
flattened_list_sentences[0]

['from',
 'river',
 'crossing',
 'to',
 'end',
 'of',
 'triäü',
 'i',
 '^',
 'pi',
 'a^h',
 '"',
 'pclcr',
 'antill',
 'ploughed',
 'deep',
 'into',
 'paddy',
 "'s",
 'river',
 'in',
 'his',
 'chrysler',
 'plymouth',
 'jr',
 'la',
 'jil',
 '?',
 'during',
 '{',
 '|',
 ')',
 'c',
 'elimination',
 'section',
 '.']

In [39]:
embedding_model_ocr = copy.deepcopy(embedding_model_orig)

In [40]:
embedding_model_ocr.workers = 8
embedding_model_ocr.vocabulary.min_count = 5
embedding_model_ocr.alpha = min_alpha_yet_reached*10.

In [41]:
embedding_model_ocr.build_vocab(flattened_list_sentences, update=True)
embedding_model_ocr.train(flattened_list_sentences, 
                          total_examples=embedding_model_ocr.corpus_count,
                          epochs=w2v_args.epochs,  
                          compute_loss=w2v_args.compute_loss)

2019-11-20 09:43:16,592 : INFO : collecting all words and their counts
2019-11-20 09:43:16,593 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 09:43:16,697 : INFO : PROGRESS: at sentence #10000, processed 486420 words, keeping 71569 word types
2019-11-20 09:43:16,778 : INFO : PROGRESS: at sentence #20000, processed 818079 words, keeping 102132 word types
2019-11-20 09:43:16,859 : INFO : PROGRESS: at sentence #30000, processed 1228071 words, keeping 139597 word types
2019-11-20 09:43:16,922 : INFO : PROGRESS: at sentence #40000, processed 1547465 words, keeping 163436 word types
2019-11-20 09:43:17,010 : INFO : PROGRESS: at sentence #50000, processed 1901060 words, keeping 187025 word types
2019-11-20 09:43:17,082 : INFO : PROGRESS: at sentence #60000, processed 2249261 words, keeping 212735 word types
2019-11-20 09:43:17,153 : INFO : PROGRESS: at sentence #70000, processed 2602920 words, keeping 237265 word types
2019-11-20 09:43:17,233 : INFO : PR

2019-11-20 09:44:05,151 : INFO : EPOCH 1 - PROGRESS: at 73.37% examples, 129818 words/s, in_qsize 16, out_qsize 1
2019-11-20 09:44:06,206 : INFO : EPOCH 1 - PROGRESS: at 75.64% examples, 129926 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:07,219 : INFO : EPOCH 1 - PROGRESS: at 78.16% examples, 130165 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:08,245 : INFO : EPOCH 1 - PROGRESS: at 80.10% examples, 130170 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:09,303 : INFO : EPOCH 1 - PROGRESS: at 82.13% examples, 130311 words/s, in_qsize 16, out_qsize 1
2019-11-20 09:44:10,332 : INFO : EPOCH 1 - PROGRESS: at 83.58% examples, 130371 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:11,342 : INFO : EPOCH 1 - PROGRESS: at 85.74% examples, 130626 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:12,356 : INFO : EPOCH 1 - PROGRESS: at 88.02% examples, 130942 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:44:13,369 : INFO : EPOCH 1 - PROGRESS: at 90.41% examples, 130962 words/s,

2019-11-20 09:45:13,790 : INFO : EPOCH 2 - PROGRESS: at 79.25% examples, 104191 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:45:14,799 : INFO : EPOCH 2 - PROGRESS: at 81.24% examples, 104730 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:15,800 : INFO : EPOCH 2 - PROGRESS: at 82.38% examples, 104575 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:16,814 : INFO : EPOCH 2 - PROGRESS: at 83.41% examples, 104395 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:17,845 : INFO : EPOCH 2 - PROGRESS: at 85.59% examples, 105018 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:18,997 : INFO : EPOCH 2 - PROGRESS: at 87.12% examples, 104907 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:20,233 : INFO : EPOCH 2 - PROGRESS: at 90.07% examples, 105410 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:45:21,310 : INFO : EPOCH 2 - PROGRESS: at 92.45% examples, 106076 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:45:22,370 : INFO : EPOCH 2 - PROGRESS: at 93.77% examples, 106004 words/s,

2019-11-20 09:46:22,781 : INFO : EPOCH 3 - PROGRESS: at 92.89% examples, 118214 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:46:23,782 : INFO : EPOCH 3 - PROGRESS: at 94.17% examples, 118032 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:46:24,797 : INFO : EPOCH 3 - PROGRESS: at 95.33% examples, 118080 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:46:25,878 : INFO : EPOCH 3 - PROGRESS: at 96.68% examples, 118343 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:46:26,889 : INFO : EPOCH 3 - PROGRESS: at 98.41% examples, 118864 words/s, in_qsize 14, out_qsize 0
2019-11-20 09:46:27,257 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-11-20 09:46:27,260 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-20 09:46:27,313 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 09:46:27,351 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 09:46:27,445 : INFO : worker thread finished; awaiting 

2019-11-20 09:47:24,461 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-20 09:47:24,461 : INFO : EPOCH - 4 : training on 10460495 raw words (7196090 effective words) took 56.9s, 126536 effective words/s
2019-11-20 09:47:25,715 : INFO : EPOCH 5 - PROGRESS: at 1.12% examples, 96319 words/s, in_qsize 16, out_qsize 1
2019-11-20 09:47:26,916 : INFO : EPOCH 5 - PROGRESS: at 3.12% examples, 117997 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:47:28,087 : INFO : EPOCH 5 - PROGRESS: at 6.07% examples, 125246 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:47:29,237 : INFO : EPOCH 5 - PROGRESS: at 8.56% examples, 128821 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:47:30,297 : INFO : EPOCH 5 - PROGRESS: at 10.61% examples, 131862 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:47:31,321 : INFO : EPOCH 5 - PROGRESS: at 12.65% examples, 131681 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:47:32,390 : INFO : EPOCH 5 - PROGRESS: at 15.21% examples, 131580 words/s, in

(35981642, 52302475)

In [42]:
print("\n\n[INFO] Save the model")
embedding_model_ocr.save("./w2v_005_EM_ocr_qual_1_2.model")

2019-11-20 09:48:22,108 : INFO : saving Word2Vec object under ./w2v_005_EM_ocr_qual_1_2.model, separately None
2019-11-20 09:48:22,109 : INFO : storing np array 'vectors' to ./w2v_005_EM_ocr_qual_1_2.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 09:48:23,334 : INFO : not storing attribute vectors_norm
2019-11-20 09:48:23,335 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_ocr_qual_1_2.model.trainables.syn1neg.npy
2019-11-20 09:48:24,537 : INFO : not storing attribute cum_table
2019-11-20 09:48:25,633 : INFO : saved ./w2v_005_EM_ocr_qual_1_2.model


## Corrected

In [43]:
list_sentences = db_sentence["corrected_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 25414


In [44]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [45]:
flattened_list_sentences[0]

['from',
 'river',
 'crossing',
 'to',
 'end',
 'of',
 'trial',
 'splash',
 ':',
 'peler',
 'antill',
 'ploughed',
 'deep',
 'into',
 'paddy',
 "'s",
 'river',
 'in',
 'his',
 'chrysler',
 'plymouth',
 'during',
 'the',
 'elimination',
 'section',
 '.']

In [46]:
embedding_model_corrected = copy.deepcopy(embedding_model_orig)

In [47]:
embedding_model_corrected.workers = 8
embedding_model_corrected.vocabulary.min_count = 5
embedding_model_corrected.alpha = min_alpha_yet_reached*10.

In [48]:
embedding_model_corrected.build_vocab(flattened_list_sentences, update=True)
embedding_model_corrected.train(flattened_list_sentences, 
                                total_examples=embedding_model_corrected.corpus_count,
                                epochs=w2v_args.epochs,  
                                compute_loss=w2v_args.compute_loss)

2019-11-20 09:48:37,484 : INFO : collecting all words and their counts
2019-11-20 09:48:37,485 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 09:48:37,541 : INFO : PROGRESS: at sentence #10000, processed 236636 words, keeping 21073 word types
2019-11-20 09:48:37,598 : INFO : PROGRESS: at sentence #20000, processed 482228 words, keeping 31622 word types
2019-11-20 09:48:37,665 : INFO : PROGRESS: at sentence #30000, processed 740659 words, keeping 39961 word types
2019-11-20 09:48:37,721 : INFO : PROGRESS: at sentence #40000, processed 988781 words, keeping 46057 word types
2019-11-20 09:48:37,770 : INFO : PROGRESS: at sentence #50000, processed 1239629 words, keeping 51414 word types
2019-11-20 09:48:37,827 : INFO : PROGRESS: at sentence #60000, processed 1498330 words, keeping 56156 word types
2019-11-20 09:48:37,873 : INFO : PROGRESS: at sentence #70000, processed 1736066 words, keeping 61054 word types
2019-11-20 09:48:37,947 : INFO : PROGRESS: 

2019-11-20 09:49:07,181 : INFO : EPOCH 1 - PROGRESS: at 44.13% examples, 132585 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:08,210 : INFO : EPOCH 1 - PROGRESS: at 46.38% examples, 132853 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:09,296 : INFO : EPOCH 1 - PROGRESS: at 48.52% examples, 132684 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:10,328 : INFO : EPOCH 1 - PROGRESS: at 50.69% examples, 132985 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:49:11,420 : INFO : EPOCH 1 - PROGRESS: at 52.62% examples, 132689 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:12,467 : INFO : EPOCH 1 - PROGRESS: at 54.71% examples, 133313 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:13,545 : INFO : EPOCH 1 - PROGRESS: at 56.84% examples, 133277 words/s, in_qsize 14, out_qsize 1
2019-11-20 09:49:14,598 : INFO : EPOCH 1 - PROGRESS: at 58.86% examples, 133184 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:49:15,617 : INFO : EPOCH 1 - PROGRESS: at 60.90% examples, 133264 words/s,

2019-11-20 09:50:15,579 : INFO : EPOCH 2 - PROGRESS: at 70.73% examples, 124977 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:16,702 : INFO : EPOCH 2 - PROGRESS: at 72.40% examples, 124223 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:17,732 : INFO : EPOCH 2 - PROGRESS: at 73.91% examples, 123607 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:18,816 : INFO : EPOCH 2 - PROGRESS: at 75.49% examples, 123194 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:19,881 : INFO : EPOCH 2 - PROGRESS: at 77.30% examples, 123006 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:20,882 : INFO : EPOCH 2 - PROGRESS: at 78.82% examples, 122814 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:21,890 : INFO : EPOCH 2 - PROGRESS: at 80.63% examples, 122944 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:50:22,952 : INFO : EPOCH 2 - PROGRESS: at 82.10% examples, 122197 words/s, in_qsize 15, out_qsize 1
2019-11-20 09:50:24,015 : INFO : EPOCH 2 - PROGRESS: at 83.52% examples, 121454 words/s,

2019-11-20 09:51:25,649 : INFO : EPOCH 3 - PROGRESS: at 89.90% examples, 119765 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:51:26,659 : INFO : EPOCH 3 - PROGRESS: at 91.94% examples, 120148 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:51:27,735 : INFO : EPOCH 3 - PROGRESS: at 94.10% examples, 120649 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:51:28,779 : INFO : EPOCH 3 - PROGRESS: at 96.17% examples, 121080 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:51:29,935 : INFO : EPOCH 3 - PROGRESS: at 98.55% examples, 121504 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:51:30,564 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-11-20 09:51:30,575 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-20 09:51:30,617 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-20 09:51:30,651 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-20 09:51:30,696 : INFO : worker thread finished; awaiting 

2019-11-20 09:52:27,435 : INFO : EPOCH 5 - PROGRESS: at 6.39% examples, 132306 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:28,472 : INFO : EPOCH 5 - PROGRESS: at 8.46% examples, 136512 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:29,552 : INFO : EPOCH 5 - PROGRESS: at 10.07% examples, 130611 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:52:30,570 : INFO : EPOCH 5 - PROGRESS: at 12.14% examples, 133146 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:31,570 : INFO : EPOCH 5 - PROGRESS: at 14.07% examples, 134368 words/s, in_qsize 16, out_qsize 0
2019-11-20 09:52:32,589 : INFO : EPOCH 5 - PROGRESS: at 16.21% examples, 135633 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:33,598 : INFO : EPOCH 5 - PROGRESS: at 18.39% examples, 136811 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:34,630 : INFO : EPOCH 5 - PROGRESS: at 20.16% examples, 136274 words/s, in_qsize 15, out_qsize 0
2019-11-20 09:52:35,701 : INFO : EPOCH 5 - PROGRESS: at 21.77% examples, 134148 words/s, i

(35235193, 51581785)

In [49]:
print("\n\n[INFO] Save the model")
embedding_model_corrected.save("./w2v_005_EM_corr_qual_1_2.model")

2019-11-20 09:53:15,879 : INFO : saving Word2Vec object under ./w2v_005_EM_corr_qual_1_2.model, separately None
2019-11-20 09:53:15,880 : INFO : storing np array 'vectors' to ./w2v_005_EM_corr_qual_1_2.model.wv.vectors.npy




[INFO] Save the model


2019-11-20 09:53:17,260 : INFO : not storing attribute vectors_norm
2019-11-20 09:53:17,261 : INFO : storing np array 'syn1neg' to ./w2v_005_EM_corr_qual_1_2.model.trainables.syn1neg.npy
2019-11-20 09:53:18,450 : INFO : not storing attribute cum_table
2019-11-20 09:53:19,527 : INFO : saved ./w2v_005_EM_corr_qual_1_2.model
